## Part 1: Preprocessing

In [136]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [137]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [138]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [139]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Age', 'Education', 'HourlyRate', 'JobSatisfaction', 'NumCompaniesWorked', 'TotalWorkingYears', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion']

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes

,0
Age,int64
Education,int64
HourlyRate,int64
JobSatisfaction,int64
NumCompaniesWorked,int64
TotalWorkingYears,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsInCurrentRole,int64
YearsSinceLastPromotion,int64


In [140]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [94]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

In [141]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the training and testing data
X_test_scaled = scaler.transform(X_test)


In [96]:
y_train_department = y_train['Department'].to_numpy().reshape(-1, 1)
y_train_attrition = y_train['Attrition'].to_numpy().reshape(-1, 1)

y_test_department = y_test['Department'].to_numpy().reshape(-1, 1)
y_test_attrition = y_test['Attrition'].to_numpy().reshape(-1, 1)

In [142]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder

department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
y_train_encoded = department_encoder.fit_transform(y_train_department)


# Create two new variables by applying the encoder
# to the training and testing data
y_test_encoded = department_encoder.transform(y_test_department)
print(y_train_encoded.shape)

(1102, 3)


In [145]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
y_train_attrition_encoded = attrition_encoder.fit_transform(y_train[['Attrition']])


# Create two new variables by applying the encoder
# to the training and testing data
y_test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']])

print(y_train_attrition_encoded.shape)

(1102, 2)


## Create, Compile, and Train the Model

In [146]:
# Find the number of columns in the X training data
num_columns_X_train = X_train.shape[1]

# Create the input layer
from tensorflow.keras.layers import Input
input_layer = Input(shape=(X_train_scaled.shape[1],))

# Create at least two shared layers
from keras.layers import Dense

shared_layer_1 = Dense(64, activation='relu')(input_layer)
shared_layer_2 = Dense(32, activation='relu')(shared_layer_1)


In [147]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_branch = Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
department_output = Dense(y_train_encoded.shape[1], activation='softmax', name='department_output')(department_branch)



In [148]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_branch = Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
attrition_output = Dense(y_train_attrition_encoded.shape[1], activation='softmax', name='attrition_output')(attrition_branch)


In [149]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss=['categorical_crossentropy', 'categorical_crossentropy'],  # Losses for both outputs
              metrics=[['accuracy'], ['accuracy']])

# Summarize the model
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_32 (Dense)          │ (None, 64)             │            704 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_33 (Dense)          │ (None, 32)             │          2,080 │ dense_32[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_34 (Dense)          │ (None, 16)             │            528 │ dense_33[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_35 (Dense)          │ (None, 16)             │            528 │ dense_33[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             51 │ dense_34[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             34 │ dense_35[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,925 (15.33 KB)

 Trainable params: 3,925 (15.33 KB)

 Non-trainable params: 0 (0.00 B)

In [150]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train_attrition_encoded:", y_train_attrition_encoded.shape)
print("Shape of y_train_encoded:", y_train_encoded.shape)


Shape of X_train: (1102, 10)
Shape of y_train_attrition_encoded: (1102, 2)
Shape of y_train_encoded: (1102, 3)


In [151]:
print(y_train['Attrition'].unique())


['No' 'Yes']


In [152]:
# Train the model
# Train the model
history = model.fit(X_train_scaled,
                    [y_train_encoded, y_train_attrition_encoded],  # List of targets for both output layers
                    validation_data=(X_test_scaled, [y_test_encoded, y_test_attrition_encoded]),  # Validation data
                    epochs=10,  # Number of epochs, you can adjust as needed
                    batch_size=32)  # You can also adjust the batch size



Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - attrition_output_accuracy: 0.4608 - department_output_accuracy: 0.3978 - loss: 1.7472 - val_attrition_output_accuracy: 0.8696 - val_department_output_accuracy: 0.6522 - val_loss: 1.3403
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - attrition_output_accuracy: 0.8209 - department_output_accuracy: 0.6285 - loss: 1.3357 - val_attrition_output_accuracy: 0.8696 - val_department_output_accuracy: 0.6522 - val_loss: 1.2042
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8369 - department_output_accuracy: 0.6626 - loss: 1.2344 - val_attrition_output_accuracy: 0.8696 - val_department_output_accuracy: 0.6522 - val_loss: 1.1936
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8330 - department_output_accuracy: 0.6477 - loss: 1.1833 - val_attrition_output_accuracy: 0.8696 - val_department_output_accuracy: 0.6522 - val_loss: 1.1906
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/st

In [153]:
# Evaluate the model with the testing data
evaluation_results = model.evaluate(X_test_scaled,
                                     [y_test_encoded, y_test_attrition_encoded],  # List of targets for both output layers
                                     verbose=1)
print("Evaluation Results:", evaluation_results)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8421 - department_output_accuracy: 0.6529 - loss: 1.2078 
Evaluation Results: [1.1938261985778809, 0.8478260636329651, 0.654891312122345]


In [154]:
# Print the accuracy for both department and attrition
loss, accuracy_attrition, accuracy_department = evaluation_results

print(f"Attrition accuracy: {accuracy_attrition:.4f}")
print(f"Department accuracy: {accuracy_department:.4f}")

Attrition accuracy: 0.8478
Department accuracy: 0.6549


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. accuracy might not be the best metric because the classes are imbalanced in Department and Attrition
2. I chose softmax because this is a multi-class classification problem
3. adjusting the hyperparameters like the number of epochs might help in improving this model as well as adding more layers